In [ ]:
#Author: Adrian J 2022-05
import os
import pytz
import sys
import shutil
from datetime import datetime
from delta import * #to-do remove the wild import from delta
from pyspark.sql import SparkSession 
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, StructType, StructField, StringType, TimestampType, IntegerType
from etl_utils.table_schema import TableSchema
from etl_utils.general_utils import persist_table

In [ ]:
## recreate cassandra required keyspace and tables
%run '/home/jovyan/work/init_script_cassandra.py'

In [ ]:
#get or create spark delta session
builder = (
    SparkSession
    .builder
    .master("local")
    .appName("p0_create_tables")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#Writing empty session_event
table_name = 'r_session_events'
schema_root_path = '/home/jovyan/work/data_lake/raw/schemas'
schema = 'assignment_data'
dl_raw_path = f'/home/jovyan/work/data_lake/raw/{schema}/{table_name}/'
partition_key = "EVENT_DATE"
dl_insert_ts_column = "DL_INSERT_TS"
sort_column = "EVENT_DATE"

# Create an empty RDD with expected schema
emp_RDD = spark.sparkContext.emptyRDD()
columns = TableSchema(table_name, schema_root_path).load_schema_json()
df_session_events_empty = spark.createDataFrame(data = emp_RDD, schema = columns)

if os.path.exists(dl_raw_path):
    print(f"INFO: Table {table_name} exists... deleting")
    try:
        shutil.rmtree(dl_raw_path)
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
    persist_table(spark, df_session_events_empty, dl_raw_path, partition_key, table_name)
else:
    persist_table(spark, df_session_events_empty, dl_raw_path, partition_key, table_name)

In [ ]:
#writing completed_session_event
table_name = 'uc_delta_session_events'
schema_root_path = '/home/jovyan/work/data_lake/use_case/schemas'
schema = 'uc_assignment'
dl_uc_path = f'/home/jovyan/work/data_lake/use_case/{schema}/{table_name}/'
partition_key = "EVENT_DATE"
dl_insert_ts_column = "DL_INSERT_TS"

# Create an empty RDD with expected schema
emp_RDD = spark.sparkContext.emptyRDD()
columns = TableSchema(table_name, schema_root_path).load_schema_json()
df_session_events_empty = spark.createDataFrame(data = emp_RDD, schema = columns)

if os.path.exists(dl_uc_path):
    print(f"INFO: Table {table_name} exists... deleting")
    try:
        shutil.rmtree(dl_uc_path)
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
    persist_table(spark, df_session_events_empty, dl_uc_path, partition_key, table_name)
else:
    persist_table(spark, df_session_events_empty, dl_uc_path, partition_key, table_name)

In [ ]:
spark.stop()